In [ ]:
from google.colab import files
uploaded = files.upload()

Saving cleaned_stock_prices.csv to cleaned_stock_prices.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib
import zipfile
from google.colab import files
df = pd.read_csv("cleaned_stock_prices.csv")
target = "Price"
def convert_volume(value):
    if isinstance(value, str):
        value = value.replace(",", "").strip()
        if value.endswith("M"):
            return float(value[:-1]) * 1_000_000
        elif value.endswith("K"):
            return float(value[:-1]) * 1_000
        elif value == "-" or value == "":
            return 0
        else:
            return float(value)
    return value
if "Vol." in df.columns:
    df["Vol."] = df["Vol."].apply(convert_volume)

# Convert percentages to float
def clean_percentage(x):
    if isinstance(x, str):
        x = x.replace("%", "").strip()
        return float(x) / 100
    return x
if "Change %" in df.columns:
    df["Change %"] = df["Change %"].apply(clean_percentage)
if "Date" in df.columns:
    df = df.drop(columns=["Date"])
X = df.drop(columns=[target])
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
ann = Sequential([
    Dense(64, activation='relu', input_dim=X_train.shape[1]),
    Dense(32, activation='relu'),
    Dense(1)
])
ann.compile(optimizer='adam', loss='mse')
history = ann.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=1)
y_pred_ann = ann.predict(X_test)
ann_mse = mean_squared_error(y_test, y_pred_ann)
ann_r2 = r2_score(y_test, y_pred_ann)
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
lr_mse = mean_squared_error(y_test, y_pred_lr)
lr_r2 = r2_score(y_test, y_pred_lr)
rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
rf_mse = mean_squared_error(y_test, y_pred_rf)
rf_r2 = r2_score(y_test, y_pred_rf)
results = pd.DataFrame({
    "Model": ["Linear Regression", "Random Forest", "ANN"],
    "MSE": [lr_mse, rf_mse, ann_mse],
    "R2 Score": [lr_r2, rf_r2, ann_r2]
})
print("\n=== MODEL PERFORMANCE COMPARISON ===")
print(results)
with zipfile.ZipFile("regression_outputs.zip", "w") as zipf:
    results.to_csv("regression_model_results.csv", index=False)
    zipf.write("regression_model_results.csv")
    joblib.dump(lr, "linear_regression_model.pkl")
    zipf.write("linear_regression_model.pkl")
    joblib.dump(rf, "random_forest_model.pkl")
    zipf.write("random_forest_model.pkl")
    ann.save("ann_model.h5")
    zipf.write("ann_model.h5")

# Download zip in Colab
files.download("regression_outputs.zip")


Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - loss: 174413.6875 - val_loss: 172688.7344
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 174644.1250 - val_loss: 172537.8125
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 174313.0625 - val_loss: 172379.8281
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 174685.5156 - val_loss: 172210.7188
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 175047.7188 - val_loss: 172017.5000
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 173034.8125 - val_loss: 171792.1250
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 174909.4375 - val_loss: 171532.7344
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 172340.9375 - val_loss: 171233.1719
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 172348.1406 - val_loss: 170886.1094
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 173171.3906 - val_loss: 170483.3281
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 171974.5625 - val_loss


=== MODEL PERFORMANCE COMPARISON ===
               Model           MSE    R2 Score
0  Linear Regression      3.581245    0.991621
1      Random Forest      8.409893    0.980323
2                ANN  56826.051902 -131.960111


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>